# ETL RAW -> SILVER | SINISTROS PRF

## 1. Configuração e Importações

Nesta etapa configuramos o ambiente do notebook, importamos bibliotecas e definimos os caminhos da camada **raw** e **silver**.  
Também carregamos as variáveis do banco via arquivo `.env`.


In [47]:
from pathlib import Path
from dotenv import load_dotenv
import os
import unicodedata

import numpy as np
import pandas as pd

import psycopg2
from psycopg2.extras import execute_values


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 80)

BASE_PATH = Path(os.getcwd()).parent.parent
RAW_PATH = BASE_PATH / "data_layer" / "raw"
SILVER_PATH = BASE_PATH / "data_layer" / "silver" / "data"
SILVER_PATH.mkdir(parents=True, exist_ok=True)

RAW_FILES = sorted([p for p in RAW_PATH.iterdir() if p.suffix.lower() == ".csv"])

print(f"Total de {len(RAW_FILES)} arquivos encontrados em raw: {', '.join([p.name for p in RAW_FILES])}")

load_dotenv(BASE_PATH / ".env")

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "database": os.getenv("POSTGRES_DB"),
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"),
}

print("DB host:", DB_CONFIG["host"])
print("DB port:", DB_CONFIG["port"])
print("DB name:", DB_CONFIG["database"])


Total de 2 arquivos encontrados em raw: acidentes2024_todas_causas_tipos.csv, acidentes2025_todas_causas_tipos.csv
DB host: localhost
DB port: 5432
DB name: prf


## 2. Funções Auxiliares (Limpeza e Conversões)

Aqui criamos funções reutilizáveis para padronizar texto, tratar valores nulos, converter tipos e validar campos importantes (UF, coordenadas, etc.).  
Essas funções deixam o `transform` mais limpo e fácil de manter.


In [ ]:
NULL_LIKE = {
    "", " ", "null", "none", "nan", "na", "n/a",
    "(null)", "nonetype", "nan", "null", "n/a"
}

UNKNOWN_LIKE = {
    "ignorado",
    "nao informado",
    "nao-informado",
    "sem informacao",
    "sem-informacao",
    "desconhecido",
    "0",
}

VALID_UF = {
    "AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MT","MS","MG",
    "PA","PB","PR","PE","PI","RJ","RN","RS","RO","RR","SC","SP","SE","TO"
}

SILVER_COLUMNS = [
    "ano_arquivo", "sinistro_id", "pessoa_id", "veiculo_id",
    "data_hora", "dia_semana_num", "uf", "municipio", "delegacia",
    "latitude", "longitude", "causa_acidente", "tipo_acidente",
    "fase_dia", "sentido_via", "condicao_meteorologica", "tipo_pista",
    "tracado_via", "caracteristicas_via", "tipo_envolvido",
    "estado_fisico", "faixa_etaria", "sexo", "tipo_veiculo",
    "faixa_idade_veiculo",
]

def normalize_scalar(x) -> str | None:
    if x is None or pd.isna(x):
        return None
    return (
        unicodedata.normalize("NFKD", str(x))
        .encode("ascii", "ignore")
        .decode("ascii")
        .lower()
        .strip()
    )


def normalize_text(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(dtype="string")

    s = s.astype("string")
    s = s.map(lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x)
    s = s.str.strip().str.replace(r"\s+", " ", regex=True)

    normalized = s.map(normalize_scalar)
    return s.mask(normalized.isin(NULL_LIKE), pd.NA)

def to_int(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(dtype="Int64")
    return pd.to_numeric(s, errors="coerce").astype("Int64")


def to_float(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(dtype="Float64")
    s = s.astype("string").str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce").astype("Float64")


def validate_coordinates(
    lat: pd.Series,
    lon: pd.Series
) -> tuple[pd.Series, pd.Series]:
    lat = lat.where(lat.isna() | lat.between(-90, 90), pd.NA)
    lon = lon.where(lon.isna() | lon.between(-180, 180), pd.NA)
    return lat.astype("Float64"), lon.astype("Float64")


def validate_uf(s: pd.Series) -> pd.Series:
    s = normalize_text(s).str.upper()
    return s.where(s.isin(VALID_UF), pd.NA).astype("string")


def null_if_unknown(s: pd.Series) -> pd.Series:
    s = normalize_text(s)

    def resolve(v):
        if v is None or pd.isna(v):
            return pd.NA
        return pd.NA if normalize_scalar(v) in UNKNOWN_LIKE else v

    return s.map(resolve).astype("string")


In [ ]:

def normalize_series(s: pd.Series) -> pd.Series:
    return normalize_text(s).map(normalize_scalar)


def parse_time(s: pd.Series) -> pd.Series:
    s = normalize_text(s)
    return (
        pd.to_datetime(s, format="%H:%M:%S", errors="coerce")
        .fillna(pd.to_datetime(s, format="%H:%M", errors="coerce"))).dt.time


DAY_MAP = {
    "segunda-feira": 0,
    "terca-feira": 1,
    "terça-feira": 1,
    "quarta-feira": 2,
    "quinta-feira": 3,
    "sexta-feira": 4,
    "sabado": 5,
    "sábado": 5,
    "domingo": 6,
}

def map_weekday(s: pd.Series) -> pd.Series:
    return normalize_series(s).map(DAY_MAP).astype("Int64")


GENDER_MAP = {
    "masculino": {"m", "masc", "masculino"},
    "feminino": {"f", "fem", "feminino"},
}

def map_gender(s: pd.Series) -> pd.Series:
    s = normalize_series(s)

    def resolve(v):
        for gender, values in GENDER_MAP.items():
            if v in values:
                return gender
        return pd.NA

    return s.map(resolve).astype("string")



PHYSICAL_STATE_RULES = [
    ("obito", {"obito", "morto"}),
    ("grave", {"grave"}),
    ("leve", {"leve"}),
    ("ileso", {"ileso", "sem ferimentos"}),
]

def map_physical_state(s: pd.Series) -> pd.Series:
    s = normalize_series(s)

    def resolve(v):
        if v is None:
            return pd.NA
        for state, keywords in PHYSICAL_STATE_RULES:
            if any(k in v for k in keywords):
                return state
        return pd.NA

    return s.map(resolve).astype("string")


LAND_USE_MAP = {
    "sim": "urbano",
    "nao": "rural",
}

def map_land_use(s: pd.Series) -> pd.Series:
    return normalize_series(s).map(LAND_USE_MAP).astype("string")


In [ ]:

# Idade para faixas.
def age_bucket(age_s: pd.Series) -> pd.Series:
    if age_s is None:
        return pd.Series([], dtype="string")
    age = pd.to_numeric(age_s, errors="coerce")
    age = age.mask((age <= 0) | (age > 120), np.nan)

    bins = [-0.1, 9, 19, 29, 39, 49, 59, 69, 79, 89, 99, 10_000]
    labels = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80-89","90-99","100+"]

    return pd.cut(age, bins=bins, labels=labels).astype("string")

# Ano fabricação para idade do veículo e depois para faixas
def vehicle_age_bucket(year_fab_s: pd.Series, year_ref_s: pd.Series) -> pd.Series:
    if year_fab_s is None or year_ref_s is None:
        return pd.Series([], dtype="string")
    year_fab = pd.to_numeric(year_fab_s, errors="coerce")
    year_ref = pd.to_numeric(year_ref_s, errors="coerce")

    age = (year_ref - year_fab).mask(lambda x: (x < 0) | (x > 120), np.nan)

    bins = [-0.1, 4, 9, 14, 19, 29, 120]
    labels = ["0-4","5-9","10-14","15-19","20-29","30+"]

    return pd.cut(age, bins=bins, labels=labels).astype("string")


## 3. Funções de Banco (Load no PostgreSQL)

Nesta etapa criamos as funções que conectam no PostgreSQL e fazem o carregamento na tabela `silver.sinistros`.  
O carregamento usa `execute_values` para inserir em lote e suportar os modos **truncate** (recarregar tudo) ou **upsert** (atualizar quando repetir PK).


In [51]:
DB_SCHEMA = "silver"
TABLE_NAME = "sinistros"
TABLE_FULL_NAME = f"{DB_SCHEMA}.{TABLE_NAME}"

def get_conn(db_config: dict):
    return psycopg2.connect(
        host=db_config["host"],
        port=db_config["port"],
        dbname=db_config["database"],
        user=db_config["user"],
        password=db_config["password"],
    )

def get_row_count(cur) -> int:
    cur.execute(f"SELECT COUNT(*) FROM {TABLE_FULL_NAME};")
    return cur.fetchone()[0]

def ensure_schema_and_table(cur):
    ddl_path = BASE_PATH / "data_layer" / "silver" / "ddl.sql"
    
    if not ddl_path.exists():
        print(f"Arquivo DDL não encontrado: {ddl_path}")
        return
    
    try:
        ddl = ddl_path.read_text(encoding="utf-8")
        cur.execute(ddl)
        print("Estrutura do banco criada com sucesso")
    except Exception as e:
        print(f"Erro ao executar DDL: {e}")

def load_to_postgres(df: pd.DataFrame, db_config: dict, mode: str = "truncate"):
    df_load = df[SILVER_COLUMNS].copy()
    df_load = df_load.astype(object).where(pd.notna(df_load), None)
    records = [tuple(row) for row in df_load.itertuples(index=False, name=None)]

    if not records:
        print("Nenhum registro para carregar (DataFrame vazio)")
        return

    print(f"Total de linhas a serem carregadas: {len(records):,}")
    
    insert_sql = f"""
        INSERT INTO {TABLE_FULL_NAME} ({",".join(SILVER_COLUMNS)})
        VALUES %s
    """

    conn = get_conn(db_config)
    cur = conn.cursor()

    try:
        ensure_schema_and_table(cur)
        conn.commit()
        try:
            count_before = get_row_count(cur)
            print(f"Registros no banco antes da carga: {count_before:,}")
        except:
            count_before = 0
            print("Tabela ainda não existe ou está vazia")

        # Truncar se necessário
        if mode == "truncate":
            cur.execute(f"TRUNCATE TABLE {TABLE_FULL_NAME};")
            conn.commit()
        
        print("⬆️  Inserindo dados...")
        execute_values(cur, insert_sql, records, page_size=5000)
        conn.commit()
        
        count_after = get_row_count(cur)
        print(f"Carga concluída com sucesso!")
        print(f"Registros no banco depois da carga: {count_after:,}")
    except Exception as e:
        conn.rollback()
        print(f"Erro durante a carga: {e}")
        raise
    finally:
        cur.close()
        conn.close()
        print(f"{'='*80}\n")


## 4. Carregando os Dados Raw

Agora carregamos todos os arquivos CSV da pasta `raw` e unimos em um único DataFrame.  
Também exibimos o tamanho final para termos noção do volume de dados.


In [52]:
# Carrega e junta os CSVs raw

def load_raw_csvs(csv_paths: list[Path]) -> pd.DataFrame:
    dfs = []
    for p in csv_paths:
        df = pd.read_csv(
            p,
            sep=";",
            encoding="ISO-8859-1",
            low_memory=False,
            dtype=str,
        )
        df["__source_file"] = p.name
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

print("\nCarregando dados Raw...")
df_raw = load_raw_csvs(RAW_FILES)

print(f"Carregado: {df_raw.shape[0]:,} linhas x {df_raw.shape[1]:,} colunas")
print("Colunas:", list(df_raw.columns))



Carregando dados Raw...
Carregado: 1,131,595 linhas x 38 colunas
Colunas: ['id', 'pesid', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_principal', 'causa_acidente', 'ordem_tipo_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'id_veiculo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude', 'regional', 'delegacia', 'uop', '__source_file']


## 5. Transformação (RAW -> SILVER)

Nesta etapa aplicamos as regras de limpeza e padronização:

- Normalização de strings e tratamento de valores inválidos  
- Conversão de IDs e remoção de registros sem chave primária  
- Criação de `data_hora` e `ano_arquivo`  
- Validação de UF e coordenadas  
- Criação de features derivadas (faixa etária, idade do veículo, etc.)  
- Deduplicação pela chave `(sinistro_id, pessoa_id)`


In [53]:


def to_silver(df: pd.DataFrame) -> pd.DataFrame:
    print("\nINICIANDO TRANSFORM (RAW -> SILVER)")
    print(f"Shape inicial: {df.shape}")

    df = df.copy()

    # Normalização geral
    print("Normalizando texto...")
    for col in df.columns:
        df[col] = normalize_text(df[col])

    # Corrigir typo de coluna
    if "condicao_metereologica" in df.columns:
        df = df.rename(columns={"condicao_metereologica": "condicao_meteorologica"})

    df["condicao_meteorologica"] = null_if_unknown(df.get("condicao_meteorologica"))

    # IDs
    print("Convertendo IDs...")
    df["sinistro_id"] = to_int(df["id"]) if "id" in df.columns else pd.NA
    df["pessoa_id"] = to_int(df["pesid"]) if "pesid" in df.columns else pd.NA
    df["veiculo_id"] = to_int(df["id_veiculo"]) if "id_veiculo" in df.columns else pd.NA
    df["sinistro_id"] = df["sinistro_id"].where(df["sinistro_id"].isna() | (df["sinistro_id"] > 0), pd.NA)
    df["pessoa_id"]   = df["pessoa_id"].where(df["pessoa_id"].isna() | (df["pessoa_id"] > 0), pd.NA)
    df["veiculo_id"]  = df["veiculo_id"].where(df["veiculo_id"].isna() | (df["veiculo_id"] > 0), pd.NA)


    # Data/hora
    print("Convertendo data e horário...")
    df["date_dt"] = pd.to_datetime(df.get("data_inversa"), format="%Y-%m-%d", errors="coerce")
    df["time_dt"] = parse_time(df.get("horario"))

    print("Criando data_hora...")
    time_txt = df["time_dt"].astype("string").fillna("00:00:00")
    df["data_hora"] = pd.to_datetime(df["date_dt"].astype("string") + " " + time_txt, errors="coerce")

    df["ano_arquivo"] = df["data_hora"].dt.year.astype("Int64")
    df["dia_semana_num"] = map_weekday(df.get("dia_semana"))

    # Coordenadas
    print("Convertendo latitude/longitude...")
    df["latitude"] = to_float(df.get("latitude"))
    df["longitude"] = to_float(df.get("longitude"))
    df["latitude"], df["longitude"] = validate_coordinates(df["latitude"], df["longitude"])

    # Campos derivados
    df["caracteristicas_via"] = map_land_use(df.get("uso_solo"))
    df["sexo_condutor"] = map_gender(df.get("sexo"))
    df["estado_fisico"] = map_physical_state(df.get("estado_fisico"))
    df["faixa_etaria"] = age_bucket(df.get("idade"))
    df["faixa_idade_veiculo"] = vehicle_age_bucket(df.get("ano_fabricacao_veiculo"), df["ano_arquivo"])

    # UF + município
    df["uf"] = validate_uf(df.get("uf"))
    df["municipio"] = df.get("municipio").str.upper()

    # Filtro de ano
    print("Aplicando filtro ano_arquivo (2024/2025)...")
    before = len(df)
    df = df[df["ano_arquivo"].isin([2024, 2025])].copy()
    print(f"   Linhas removidas pelo filtro: {before - len(df):,}")

    # Garantir colunas finais
    print("Garantindo colunas do contrato...")
    for col in SILVER_COLUMNS:
        if col not in df.columns:
            df[col] = pd.NA
    
    before_pk = len(df)
    df = df[df["sinistro_id"].notna() & df["pessoa_id"].notna()].copy()
    print(f"   Linhas removidas por PK inválida: {before_pk - len(df):,}")


    # Deduplicação por PK
    print("Removendo duplicatas por (sinistro_id, pessoa_id)...")
    before = len(df)
    df["__completeness"] = df[SILVER_COLUMNS].notna().sum(axis=1)

    df = df.sort_values(
        ["sinistro_id", "pessoa_id", "__completeness", "data_hora"],
        ascending=[True, True, False, False],
        na_position="last",
    )

    df = df.drop_duplicates(subset=["sinistro_id", "pessoa_id"], keep="first").drop(columns="__completeness")
    print(f"   Duplicatas removidas: {before - len(df):,}")

    df_silver = df[SILVER_COLUMNS].copy()
    print(f"Shape final (silver): {df_silver.shape}")
    return df_silver

df_silver = to_silver(df_raw)
df_silver.head()



INICIANDO TRANSFORM (RAW -> SILVER)
Shape inicial: (1131595, 38)
Normalizando texto...
Convertendo IDs...
Convertendo data e horário...
Criando data_hora...
Convertendo latitude/longitude...
Aplicando filtro ano_arquivo (2024/2025)...
   Linhas removidas pelo filtro: 0
Garantindo colunas do contrato...
   Linhas removidas por PK inválida: 110,106
Removendo duplicatas por (sinistro_id, pessoa_id)...
   Duplicatas removidas: 682,306
Shape final (silver): (339183, 25)


,ano_arquivo,sinistro_id,pessoa_id,veiculo_id,data_hora,dia_semana_num,uf,municipio,delegacia,latitude,longitude,causa_acidente,tipo_acidente,fase_dia,sentido_via,condicao_meteorologica,tipo_pista,tracado_via,caracteristicas_via,tipo_envolvido,estado_fisico,faixa_etaria,sexo,tipo_veiculo,faixa_idade_veiculo
0,2024,571772,1268971,1018215,2024-01-01 00:05:00,0,RJ,TANGUA,DEL02-RJ,-22.72936,-42.701125,Reação tardia ou ineficiente do condutor,Colisão com objeto,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,urbano,Condutor,obito,20-29,Masculino,Motocicleta,0-4
1,2024,571774,1268985,1018226,2024-01-01 00:05:00,0,GO,ANAPOLIS,DEL02-GO,-16.229185,-49.009797,Animais na Pista,Colisão com objeto,Plena Noite,Decrescente,Céu Claro,Dupla,Reta,rural,Condutor,ileso,30-39,Feminino,Automóvel,15-19
3,2024,571777,1269020,1018251,2024-01-01 01:45:00,0,ES,SERRA,DEL02-ES,-20.172928,-40.267364,Reação tardia ou ineficiente do condutor,Colisão com objeto,Plena Noite,Decrescente,Nublado,Múltipla,Interseção de Vias;Reta,urbano,Condutor,ileso,50-59,Masculino,Caminhonete,15-19
4,2024,571778,1269028,1018261,2024-01-01 00:45:00,0,SC,PENHA,DEL03-SC,-26.83477,-48.706151,Reação tardia ou ineficiente do condutor,Saída de leito carroçável,Plena Noite,Crescente,Chuva,Dupla,Curva,rural,Condutor,ileso,50-59,Masculino,Camioneta,10-14
5,2024,571778,1269045,1018261,2024-01-01 00:45:00,0,SC,PENHA,DEL03-SC,-26.83477,-48.706151,Reação tardia ou ineficiente do condutor,Saída de leito carroçável,Plena Noite,Crescente,Chuva,Dupla,Curva,rural,Passageiro,leve,30-39,Feminino,Camioneta,10-14


## 6. Validações e Exportação para CSV

Antes de salvar, fazemos checagens simples para garantir qualidade:

- distribuição por ano  
- nulos nas chaves  
- valores “ignorado” que deveriam ter virado NULL  

Depois salvamos o CSV final na camada Silver.


In [54]:
print("\nVALIDAÇÕES")

print("Ano (value_counts):")
print(df_silver["ano_arquivo"].value_counts(dropna=False))

print("\nNulos nas chaves:")
print("sinistro_id null:", df_silver["sinistro_id"].isna().sum())
print("pessoa_id   null:", df_silver["pessoa_id"].isna().sum())

print("\n'ignorado' em condicao_meteorologica (ideal = 0):")
print((df_silver["condicao_meteorologica"].astype("string").str.lower() == "ignorado").sum())

OUTPUT_FILE = SILVER_PATH / "sinistros_silver.csv"
df_silver.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

print("\nCSV Silver salvo em:", OUTPUT_FILE)
print("Tamanho (linhas):", len(df_silver))



VALIDAÇÕES
Ano (value_counts):
ano_arquivo
2024    179114
2025    160069
Name: count, dtype: Int64

Nulos nas chaves:
sinistro_id null: 0
pessoa_id   null: 0

'ignorado' em condicao_meteorologica (ideal = 0):
0

CSV Silver salvo em: /Users/caiorocha/Workspace/unb/SBD2-Grupo-20-PRF/data_layer/silver/data/sinistros_silver.csv
Tamanho (linhas): 339183


## 7. Carga no PostgreSQL (silver.sinistros)

Por fim, carregamos o dataset processado no banco PostgreSQL.  
O modo `truncate` apaga tudo e recarrega do zero.  
Depois validamos conferindo o total de registros na tabela.


In [ ]:
print("\nCARREGANDO NO POSTGRES (silver.sinistros)...")


# Mode:"truncate" = limpa e recarrega tudo, "upsert"   = insere/atualiza se repetir PK
load_to_postgres(df_silver, DB_CONFIG, mode="truncate")



CARREGANDO NO POSTGRES (silver.sinistros)...
Total de linhas a serem carregadas: 339,183
Estrutura do banco criada com sucesso
Registros no banco antes da carga: 0
⬆️  Inserindo dados...
Carga concluída com sucesso!
Registros no banco depois da carga: 339,183

Total no banco: 339,183
